# Customer Analytics [ CLV Nigros Project ]
__Eray Ates, Sibel Gurbuz__

## Import database connection libraries

In [2]:
from library.libx import *
import pandas as pd

## Set connection parameters

In [3]:
config = jsonread("config.json")
con_config = config["connection"]

migros = MySQL(
    con_config["username"],
    con_config["password"],
    con_config["URL"],
    con_config["port"],
    con_config["db"]
    )

## Set calculation for how many user

In [4]:
user_count = 50

In [5]:
# Customer average spend money from a few random users
ResultProxy = migros.connection.execute('select i.total_amount, a.id, i.invoice_date from \
    user_data as a, invoice as i \
    where i.user_id = a.id order by RAND() LIMIT {};'.format(user_count))
results = ResultProxy.fetchall()
df = pd.DataFrame(results, columns=['amount','id','date']).set_index('id').sort_index()
df = df.rename_axis("id", axis="columns").rename_axis(None)

In [6]:
df.head()

id,amount,date
382,88.42,2016-02-28 04:32:41
383,293.20,2016-10-25 04:15:14
384,57.86,2016-11-13 03:00:02
386,144.14,2019-01-05 20:04:32
386,285.59,2019-01-16 01:18:42


In [19]:
grouped = df.groupby(level=0)
grouped_avg_price = grouped['amount'].agg("mean")

In [28]:
avg_price = grouped_avg_price.values.mean(); avg_price

141.3613157894737

## How many times a customer buy something in 1 year

In [42]:
from datetime import datetime, timedelta
now = datetime.now()

In [54]:
counts = df.loc[:, ['date']][df['date']> (now + timedelta(days=-365))].groupby(level=0)['date'].agg('count')

In [57]:
counts_frame = pd.DataFrame(counts)
counts_frame.head()

,date
386,2
404,1
406,1
443,1
444,1


## Put all in one dataframe

In [62]:
clv_frame = pd.DataFrame(grouped_avg_price)

In [63]:
clv_frame = pd.merge(clv_frame, counts_frame, left_index=True, right_index=True, how='outer').fillna(0)

In [64]:
clv_frame.head()

,amount,date
382,88.420,0.0
383,293.200,0.0
384,57.860,0.0
386,214.865,2.0
389,113.930,0.0


## Average money spend per user in a year

In [65]:
clv_frame['mul'] = clv_frame['amount'] * clv_frame['date']

In [67]:
clv_frame.head()

,amount,date,mul
382,88.420,0.0,0.00
383,293.200,0.0,0.00
384,57.860,0.0,0.00
386,214.865,2.0,429.73
389,113.930,0.0,0.00


Average count all users

In [69]:
result = clv_frame['mul'].mean(); result

27.62986842105263

So each of customer spendding average above count TL in a year

If we think our customer life time is 20 year

In [70]:
result * 20

552.5973684210526

## So we can't spend more than (above count) TL above 20 year to gain a new customer.